# Automatic import of Basel drinking fountains data to Wikidata
The following script downloads fountain data from Open Data swiss as per https://github.com/water-fountains/import2wikidata/issues/12, compares it to existing fountains in Wikidata for the same region, and creates Wikidata Quickstatement commands to complete the entries in Wikidata. New entities are created if no matching fountains are found.

## Initialize environment

In [1]:
from datetime import datetime as dt
dtFmt = "%y%m%d_%H%M%S"
print (dt.now().strftime(dtFmt))
import pandas as pd
import io
import numpy as np
from urllib.request import urlopen
import json
from math import *
from platform import python_version
print("Python v "+python_version())
#https://github.com/paulhoule/gastrodon/issues/7 
from gastrodon import RemoteEndpoint,QName,ttl,URIRef,inline
from matplotlib import pyplot


191031_055456
Python v 3.6.5


In [2]:
#@prefix wikibase: <wikibase: <http://wikiba.se/ontology#> .
prefixes=inline("""
   @prefix wd: <http://www.wikidata.org/entity/> .
   @prefix wdt: <http://www.wikidata.org/prop/direct/> .
   @prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
   @prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
""").graph
endpoint=RemoteEndpoint(
   "http://query.wikidata.org/sparql"
   ,prefixes=prefixes
)

## Load data

In [3]:
df = pd.read_csv("data/Trinkbrunnen.csv")

In [4]:
df.head()

,X,Y,Name,Description
0,7.629450,47.562891,Äänisbreetli-Brunnen,"<img src=""https://doc-04-24-mymaps.googleuserc..."
1,7.597086,47.558396,Achteck-Brunnen,"<img src=""https://doc-0s-24-mymaps.googleuserc..."
2,7.593189,47.550235,Aeschengraben-Brunnen,"<img src=""https://doc-00-24-mymaps.googleuserc..."
3,7.586400,47.558155,Affen-Brunnen,"<img src=""https://doc-0g-24-mymaps.googleuserc..."
4,7.593095,47.554276,Affen-Brunnen,"<img src=""https://doc-0g-24-mymaps.googleuserc..."


### Rename columns to make them easier to work with

In [5]:

# remove not needed columns
df = df.drop(columns=['Description'])
# rename columns
df = df.rename(index=str, columns=
               {"Name": "label_de"})

## Identify already existing fountains
### Query fountains from Wikidata

In [6]:
# Find the geographic extent of the data

buffer = 0.0003  # in degrees, corresponds to about 20-30 meters)
bounds = {
    'minX': df['X'].min() - buffer,
    'minY': df['Y'].min() - buffer,
    'maxX': df['X'].max() + buffer,
    'maxY': df['Y'].max() + buffer
}

In [12]:
# Query fountains (both water wells and fountains) from Wikidata within bounding box found above

query_string = """ SELECT ?place ?placeLabel ?location ?date ?catalog_code ?catalogLabel ?operator ?water_supply_type
WHERE
{{
  # Enter coordinates
  SERVICE wikibase:box {{
    ?place wdt:P625 ?location .
    bd:serviceParam wikibase:cornerWest "Point({minX} {minY})"^^geo:wktLiteral.
    bd:serviceParam wikibase:cornerEast "Point({maxX} {maxY})"^^geo:wktLiteral.
  }} .
  # Is a water well or fountain or subclass of fountain
  FILTER (EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q43483 }} || EXISTS {{ ?place wdt:P31/wdt:P279* wd:Q483453 }}).
  SERVICE wikibase:label {{
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" .
  }} 
  OPTIONAL {{ ?place p:P528 ?catalog_code.
            ?catalog_code pq:P972 ?catalog.}}
  OPTIONAL {{ ?place wdt:P571 ?date.}}
  OPTIONAL {{ ?place wdt:P5623 ?water_supply_type}}
  OPTIONAL {{ ?place wdt:P137 ?operator.}}
}}
  """.format(**bounds)

# Perform query
query_result = endpoint.select(query_string)

In [13]:
print(query_string)
print("\n\nTotal number of rows incl. duplicates "+str(query_result.size))

 SELECT ?place ?placeLabel ?location ?date ?catalog_code ?catalogLabel ?operator ?water_supply_type
WHERE
{
  # Enter coordinates
  SERVICE wikibase:box {
    ?place wdt:P625 ?location .
    bd:serviceParam wikibase:cornerWest "Point(7.5570532 47.5279767)"^^geo:wktLiteral.
    bd:serviceParam wikibase:cornerEast "Point(7.630431 47.5840738)"^^geo:wktLiteral.
  } .
  # Is a water well or fountain or subclass of fountain
  FILTER (EXISTS { ?place wdt:P31/wdt:P279* wd:Q43483 } || EXISTS { ?place wdt:P31/wdt:P279* wd:Q483453 }).
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "[AUTO_LANGUAGE],de" .
  } 
  OPTIONAL { ?place p:P528 ?catalog_code.
            ?catalog_code pq:P972 ?catalog.}
  OPTIONAL { ?place wdt:P571 ?date.}
  OPTIONAL { ?place wdt:P5623 ?water_supply_type}
  OPTIONAL { ?place wdt:P137 ?operator.}
}
  


Total number of rows incl. duplicates 3064


### Tidy up data

In [14]:
# Extract coordinates from Wikidata results

query_result['X'] = query_result['location'].apply(lambda l:float(l.split('(')[1].split(' ')[0]))
query_result['Y'] = query_result['location'].apply(lambda l:float(l.split(' ')[1].split(')')[0]))
query_result.head(100)

,place,placeLabel,location,date,catalog_code,catalogLabel,operator,water_supply_type,X,Y
0,wd:Q61881745,Allschwilerplatz-Brunnen,Point(7.5693423 47.5574918),1933-01-01T00:00:00Z,None,None,None,None,7.569342,47.557492
1,wd:Q72935358,Antonierhof-Brunnen,Point(7.5927933 47.5595776),None,None,None,wd:Q72936279,None,7.592793,47.559578
2,wd:Q72935373,Brunnen im Bruderholzschulhaus,Point(7.595505 47.5309589),None,None,None,wd:Q72936279,None,7.595505,47.530959
3,wd:Q72935387,Gemsberg-Brunnen,Point(7.5864069 47.5564634),None,None,None,wd:Q72936279,None,7.586407,47.556463
4,wd:Q72935405,Pisoni-Brunnen,Point(7.5915487 47.5566852),None,None,None,wd:Q72936279,None,7.591549,47.556685
...,...,...,...,...,...,...,...,...,...,...
95,wd:Q72966547,Zweitrog-Brunnen,Point(7.5908755 47.5564752),None,None,None,wd:Q72936279,None,7.590876,47.556475
96,wd:Q72966562,Zwerg-Nase-Brunnen,Point(7.5816178 47.5661476),None,None,None,wd:Q72936279,None,7.581618,47.566148
97,wd:Q72966929,SchatzkÃ¤stli-Brunnen,Point(7.6123694 47.5698186),None,None,None,wd:Q72936279,None,7.612369,47.569819
98,wd:Q72966944,SchifflÃ¤nde-Brunnen,Point(7.5881343 47.5600582),None,None,None,wd:Q72936279,None,7.588134,47.560058


In [21]:
# show duplicates
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html#pandas.DataFrame.duplicated

ids = query_result['placeLabel']
duplicates = query_result[ids.isin(ids[ids.duplicated()])]
dupli = duplicates.drop(columns=['catalog_code', 'catalogLabel', 'water_supply_type', 'date', 'catalogLabel'])
dupli.head(100)

,place,placeLabel,location,operator,X,Y
0,wd:Q61881745,Allschwilerplatz-Brunnen,Point(7.5693423 47.5574918),None,7.569342,47.557492
1,wd:Q72935358,Antonierhof-Brunnen,Point(7.5927933 47.5595776),wd:Q72936279,7.592793,47.559578
2,wd:Q72935373,Brunnen im Bruderholzschulhaus,Point(7.595505 47.5309589),wd:Q72936279,7.595505,47.530959
3,wd:Q72935387,Gemsberg-Brunnen,Point(7.5864069 47.5564634),wd:Q72936279,7.586407,47.556463
4,wd:Q72935405,Pisoni-Brunnen,Point(7.5915487 47.5566852),wd:Q72936279,7.591549,47.556685
...,...,...,...,...,...,...
115,wd:Q72967254,Spielplatz-Brunnen,Point(7.5833747 47.5613677),wd:Q72936279,7.583375,47.561368
116,wd:Q72967279,Spittelmatthof-Brunnen,Point(7.6238117 47.5747461),wd:Q72936279,7.623812,47.574746
117,wd:Q72967316,St. Brigitta-Brunnen,Point(7.5981939 47.5539627),wd:Q72936279,7.598194,47.553963
118,wd:Q72967331,Steinbock-Brunnen,Point(7.5901271 47.539909),wd:Q72936279,7.590127,47.539909


In [ ]:
# remove duplicate entries

# duplicate entries are caused when e.g. a fountain has catalog codes from two catalogs
query_result = query_result.drop_duplicates('place')
print("\n\nTotal number of rows without duplicates "+str(query_result.size))

### Compute distances between fountains

In [ ]:
# helper function to compute distances on the globe, returns distances in meters
def spherical_dist(pos1, pos2, r=6371000):
    pos1 = pos1 * np.pi / 180
    pos2 = pos2 * np.pi / 180
    cos_lat1 = np.cos(pos1[..., 0])
    cos_lat2 = np.cos(pos2[..., 0])
    cos_lat_d = np.cos(pos1[..., 0] - pos2[..., 0])
    cos_lon_d = np.cos(pos1[..., 1] - pos2[..., 1])
    return r * np.arccos(cos_lat_d - cos_lat1 * cos_lat2 * (1 - cos_lon_d))


# compute distances from each ODZ fountain to each Wikidata fountain
distances = spherical_dist(df[['X','Y']].values[:, None], query_result[['X','Y']].values)

### Identify nearest and second nearest matches for each OpenData.swiss basel fountain

In [ ]:
# indexes of nearest fountains
nearest_idx = np.argmin(distances, axis=1).tolist()

# QID of nearest fountains
df['nearest_qid'] = query_result.iloc[nearest_idx]['place'].apply(lambda id:id[3:]).tolist()

# distance to nearest fountain
df['nearest_distance'] = np.min(distances, axis=1).tolist()


# then remove nearest
i_line=0
for i_col in nearest_idx:
    distances[i_line, i_col] = 100000
    i_line += 1
# find distance to second nearest
df['2nd_nearest_distance'] = np.min(distances, axis=1).tolist()

df.head(100)

### Find out what information already exists for the nearest fountains

In [ ]:
# does nearest have label in german?
df['nearest_has_label_de'] = (query_result.iloc[nearest_idx]['place'].apply(lambda p:p[3:]) != query_result.iloc[nearest_idx]['placeLabel']).tolist()

# does nearest have date?
df['nearest_has_date'] = query_result.iloc[nearest_idx]['date'].apply(lambda d:d is not None).tolist()

# does nearest have operator?
df['nearest_has_operator'] = query_result.iloc[nearest_idx]['operator'].apply(lambda id:id is not None).tolist()

# does nearest have catalog code?
df['nearest_has_code'] = query_result.iloc[nearest_idx]['catalog_code'].apply(lambda id:id is not None).tolist()

# does nearest have water type?
df['nearest_has_water_type'] = query_result.iloc[nearest_idx]['water_supply_type'].apply(lambda id:id is not None).tolist()

### Decide on whether nearest fountain should be considered a match

In [ ]:
# The nearest fountain is a match if: 
# - no further than x m away
# - 2nd nearest fountain at nearest least ratio_min further away than the nearest fountain
def validate_proposal(qid, d1, d2, dmax=10, ratio_min=0.5):
    
    if d1 == 0 or (d1<=dmax and d2/d1-1 >= ratio_min):
        return 'match'
    elif d1<=dmax and d2/d1-1 < ratio_min:
        return 'unclear'
    else:
        return 'no match'
    
for index, row in df.iterrows():
    df.loc[index, 'match_found'] = validate_proposal(
        row['nearest_qid'], 
        row['nearest_distance'], 
        row['2nd_nearest_distance'],
        dmax=15
    )
dffinal = df.drop(columns=['nearest_distance', '2nd_nearest_distance'])

In [ ]:
dffinal.head()

In [ ]:
print("\n\nTotal number of rows "+str(dffinal.size))

## Create Quickstatement commands from data
### Helper functions to format content according to Quickstatements v1 syntax

In [ ]:
def process_coordinates(x, y):
    # format geographic coordinates
    return '@{1:1.8f}/{0:1.8f}'.format(x,y)


def process_year(date):
    # format date
    if np.isnan(date):
        return ''
    else:
        return '+{0:4d}-00-00T00:00:00Z/9'.format(int(date))

    
fountain_type_map = {
    'öffentlicher Brunnen': 'Q53628296',
    'Notwasserbrunnen': 'Q53628522',
    'privater Brunnen': 'Q53629707',
    'Brunnen in städtischer Liegenschaft': 'Q53628618',
    'Brunnen des Verschönerungsvereins': 'Q53628761',
    'Brunnen mit eigener Versorgung': 'Q53630002'
}

water_type_map = {
    'Verteilnetz': 'Q53633635',
    'Quellwasser': 'Q1881858',
    'eigene Versorgung': 'Q53634173',
    'Grundwasser': 'Q161598'
}

def process_fountain_type(type):
    # translate fountain types to wikidata values
    return fountain_type_map[type]


def process_water_type(type):
    # translate water types to wikidata values
    return water_type_map[type]


def process_label_de(text):
    # process German language labels
    if text is None:
        return ''
    elif 'brunnen' in text.lower():
        return '"{}"'.format(text)
    else:
        return '"Brunnen ({})"'.format(text)
    

def createline(lines, item, prop, value, qualifiers=[]):
    # general function to create Quickstatement v1 commands
    if value != '' and value != '""':
        statement = '{}\t{}\t{}'.format(item, prop, value)
        if len(qualifiers):
            # append qualifiers if applicable
            for q in qualifiers:
                statement += '\t{}\t{}'.format(q['prop'], q['value'])
        statement += '\n'
        lines.append(statement)
    return lines

### Create statements, taking care not to overwrite existing data

In [ ]:
# initialize command storage list
lines = []

for index, row in dffinal.iterrows():
    
    # either create new or edit existing entity
    if row['match_found'] == 'no match':
        # create a new fountain
        lines.append('CREATE\n')
        item = 'LAST'
    elif row['match_found'] == 'unclear':
        print('unclear match')
        print(row)
        continue
    elif row['match_found'] == 'match':
        # update existing fountain
        item = row['nearest_qid']
        
        
    # Add this basic information only if creating a new entity
    if item == 'LAST':
        # instance of drinking fountain
        lines = createline(lines, item, 'P31', 'Q1630622')

        # coordinates
        lines = createline(lines, item, 'P625', process_coordinates(row['X'], row['Y']))
        
        # TODO put reference "https://opendata.swiss/dataset/bade-trinkwasser-und-zierbrunnen-in-basel"
        
        
    # For other properties, add information if the entity is new or if property does not yet exist
    
    # label in german
    if item == 'LAST' or not row['nearest_has_label_de']:
        lines = createline(lines, item, 'Lde', process_label_de(row['label_de']))
    
    # creation date
    #if item == 'LAST' or not row['nearest_has_date']:
    #    lines = createline(lines, item, 'P571', process_year(row['date']))

    # operated by IWB  
    lines = createline(lines, item, 'P137', 'Q72936279')
            
    # catalog number can always be added (it is hard to check for)
    #lines = createline(lines, item, 'P528', '"{}"'.format(row['operator_id']), [{
    #    'prop': 'P972',
    #    'value': 'Q53629101'
    #}])

# Write commands to file

In [ ]:
quickStatFileName = "quickstatement_commands_BS_drink_"+dt.now().strftime(dtFmt)+".txt"
with io.open(quickStatFileName, "w", encoding='utf8') as f:
    f.writelines(lines)
print("wrote '"+quickStatFileName+"' with "+str(len(lines))+" lines")

# Import into Wikidata
- Go to https://tools.wmflabs.org/wikidata-todo/quick_statements.php.
- Authenticate yourself with your Wikidata account.
- Copy and paste the contents of quickstatement_commands*.txt into the blank field, and run the commands

see ../20191030_1600_import.png

...
58. Processing Q72935495 (Q72935495 Lde "Brunnen (Seelöwe-Planschbecken )")
59. Processing Q72935495 (Q72935495 P137 Q27229237)

All done!.